In [1]:
# Prepare datasets
# ddf = dd.read_csv(r"C:\Users\marij\Documents\GitHub\732A76_Research_Project\Data\HIGSS.csv")

# ddf.loc[:40000, :].to_parquet(r"C:\Users\marij\Documents\GitHub\732A76_Research_Project\Data\small.parquet")
# ddf.to_parquet(r"C:\Users\marij\Documents\GitHub\732A76_Research_Project\Data\medium.parquet")
# dd.concat([ddf, ddf]).to_parquet(r"C:\Users\marij\Documents\GitHub\732A76_Research_Project\Data\large.parquet")

In [18]:
import dask.dataframe as dd
import numpy as np
from dask_ml.linear_model import LinearRegression, LogisticRegression
from dask_ml.xgboost import XGBClassifier
from dask_ml.cluster import KMeans
from time import perf_counter

res = np.zeros((3, 3, 10)) # Size, model, iteration
for i in range(10):
    for j, size in enumerate(["small", "medium", "large"]):
        # Prep data
        print(f"{size.capitalize()} data")
        dda = dd.read_parquet(fr"C:\Users\marij\Documents\GitHub\732A76_Research_Project\Data\{size}.parquet").repartition(npartitions=[1, 75, 175][j]).to_dask_array(lengths=True)
        print(f"{len(dda.chunks[0])} partitions")
        
        X = dda[:, 1:]
        y = dda[:, 0]

        # Linear regression
        lin_reg = LinearRegression()

        start_time = perf_counter()   
        lin_reg.fit(X, y)
        end_time = perf_counter()
        
        res[j, 0, i] = end_time - start_time
        
        # Logistic regression
        log_reg = LogisticRegression()
        
        start_time = perf_counter()
        log_reg.fit(X, y)
        end_time = perf_counter()

        res[j, 1, i] = end_time - start_time

        # Decision trees
        # xgb = XGBClassifier()

        # start_time = perf_counter()
        # xgb.fit(X, y)
        # end_time = perf_counter()

        # print(f"\tDecision trees: {end_time - start_time:.3f}")
        
        # KMeans
        kmeans = KMeans(n_clusters=2)

        start_time = perf_counter()
        kmeans.fit(X)
        end_time = perf_counter()

        res[j, 2, i] = end_time - start_time

Small data


1 partitions


c:\Users\marij\miniconda3\envs\research_project_dask\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Medium data
75 partitions


c:\Users\marij\miniconda3\envs\research_project_dask\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


...

c:\Users\marij\miniconda3\envs\research_project_dask\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Large data
175 partitions


c:\Users\marij\miniconda3\envs\research_project_dask\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Small data
1 partitions


KeyboardInterrupt: 

In [19]:
for i in range(3):
    size = ["small", "medium", "large"][i]
    print(size)
    for j in range(3):
        model = ["linear regression", "logistic regression", "k-means"][j]
        print(f"{model}: {res[i, j, :][res[i, j, :] != 0].mean():.3f}")

small
linear regression: 91.576
logistic regression: 98.618
k-means: 157.425
medium
linear regression: 1732.945
logistic regression: 1777.468
k-means: 128.901
large
linear regression: 3488.985
logistic regression: 3565.304
k-means: 251.946


In [21]:
res[i, j, :][res[i, j, :] != 0]

array([236.2924066, 257.2993148, 234.6612128, 279.5303539,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ])

1125